<a href="https://colab.research.google.com/github/herusy/Machine_Learning/blob/main/%D0%93%D0%B5%D1%80%D1%83%D1%81_%D0%BF%D1%80_7_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Герус Я.О. ФІТ 4-8

In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357283 sha256=06259cf390842415779202d8aa95389ea80ad4fef7da2bb0fae6026684b54bd8
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
from surprise import Dataset, Reader
from surprise import SVD, KNNBasic
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import accuracy
from surprise.model_selection import train_test_split
import pandas as pd
from google.colab import drive


drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Завантаження датасета для рецензій BX-Book-Ratings.csv за допомогою бібліотеки
#Surprise та виведення перших 5 рядків завантаженого датасету.

file_path = '/content/drive/My Drive/BX-Book-Ratings.csv'
df = pd.read_csv(file_path, delimiter=';', encoding='latin1', nrows=10000)

reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(df[['User-ID', 'ISBN', 'Book-Rating']], reader)

for row in data.raw_ratings[:5]:
    print(row[:3])

(276725, '034545104X', 0.0)
(276726, '0155061224', 5.0)
(276727, '0446520802', 0.0)
(276729, '052165615X', 3.0)
(276729, '0521795028', 6.0)


In [ ]:
print("Перші 5 рядків датасету з поясненням:")
for row in data.raw_ratings[:5]:
    user_id, book_id, rating, timestamp = row
    print(f"Користувач {user_id} оцінив книгу {book_id} на {rating} балів)")

Перші 5 рядків датасету з поясненням:
Користувач 276725 оцінив книгу 034545104X на 0.0 балів)
Користувач 276726 оцінив книгу 0155061224 на 5.0 балів)
Користувач 276727 оцінив книгу 0446520802 на 0.0 балів)
Користувач 276729 оцінив книгу 052165615X на 3.0 балів)
Користувач 276729 оцінив книгу 0521795028 на 6.0 балів)


In [ ]:
# Реалізація двох алгоритмів для рекомендаційної системи на основі цього
#датасету. Використання крос-валідації для підбору оптимальних параметрів для
# обох алгоритмів.

trainset, testset = train_test_split(data, test_size=0.2)
param_grid_svd = {
    'n_factors': [20, 150],
    'lr_all': [0.002, 0.008],
    'reg_all': [0.01, 0.1]
}

param_grid_knn = {
    'k': [10, 50],
    'min_k': [3, 7],
    'sim_options': {
        'name': ['msd', 'pearson'],
        'user_based': [True]
    }
}


grid_search_svd = GridSearchCV(SVD, param_grid_svd, measures=['rmse', 'mae'], cv=5)
grid_search_svd.fit(data)

grid_search_knn = GridSearchCV(KNNBasic, param_grid_knn, measures=['rmse', 'mae'], cv=5)
grid_search_knn.fit(data)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity ma

In [ ]:
best_svd = grid_search_svd.best_estimator['mae']
best_knn = grid_search_knn.best_estimator['mae']

print(f"Найкращий MAE для SVD: {grid_search_svd.best_score['mae']}")
print(f"Найкращий MAE для KNNBasic: {grid_search_knn.best_score['mae']}")

# Вибір моделі з найменшим MAE

if grid_search_svd.best_score['mae'] < grid_search_knn.best_score['mae']:
    best_model = best_svd
    print("Вибраний алгоритм: SVD")
else:
    best_model = best_knn
    print("Вибраний алгоритм: KNNBasic")

Найкращий MAE для SVD: 2.253418001544779
Найкращий MAE для KNNBasic: 2.90601032896515
Вибраний алгоритм: SVD


In [ ]:
# Виведення доступних користувачів з оригінальними ID
available_user_ids = [trainset.to_raw_uid(uid) for uid in trainset.all_users()]
print("Оригінальні ID користувачів у навчальному наборі:", available_user_ids[:100])

Оригінальні ID користувачів у навчальному наборі: [277705, 278418, 278356, 277478, 277639, 183, 277427, 277879, 277954, 277945, 277982, 278465, 277229, 276786, 278017, 278137, 278724, 277928, 278512, 276754, 278665, 242, 276946, 277195, 53, 278221, 277882, 276875, 277272, 276925, 243, 278648, 277203, 278633, 278188, 278535, 276811, 276737, 276847, 278723, 278843, 277523, 278026, 277256, 278851, 278582, 278144, 276929, 277186, 39, 277378, 278002, 277433, 276822, 276954, 278554, 276856, 278663, 277746, 277457, 277930, 278568, 277134, 278692, 277087, 277872, 278586, 277908, 178, 277918, 277168, 99, 278245, 276726, 277938, 276939, 278506, 12, 278849, 278771, 277187, 278361, 278714, 278524, 277157, 277554, 276866, 278194, 278253, 277847, 277180, 278466, 278064, 277589, 278694, 176, 278668, 277042, 278218, 277901]


In [ ]:
# Виведення 10 рекомендацій книг для конкретного користувача.

best_model.fit(trainset)

user_id = 276949

user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]
print(f"Кількість рецензій користувача {user_id}: {len(user_ratings)}")

all_items = set(trainset.all_items())
rated_items = set([item for (item, _) in user_ratings])
unrated_items = all_items - rated_items

predictions = [(item, best_model.predict(user_id, trainset.to_raw_iid(item)).est) for item in unrated_items]
predictions.sort(key=lambda x: x[1], reverse=True)

print("Топ-10 книг, рекомендованих для користувача:")
for item_id, rating in predictions[:10]:
    print(f"Книга {trainset.to_raw_iid(item_id)} з прогнозованим рейтингом {rating:.2f}")



Кількість рецензій користувача 276949: 1
Топ-10 книг, рекомендованих для користувача:
Книга 0440414121 з прогнозованим рейтингом 5.37
Книга 0142001740 з прогнозованим рейтингом 5.15
Книга 0452282152 з прогнозованим рейтингом 5.06
Книга 0394820371 з прогнозованим рейтингом 4.79
Книга 0312195516 з прогнозованим рейтингом 4.72
Книга 0671027360 з прогнозованим рейтингом 4.68
Книга 0440241073 з прогнозованим рейтингом 4.68
Книга 0440225701 з прогнозованим рейтингом 4.59
Книга 0446310786 з прогнозованим рейтингом 4.52
Книга 0767902890 з прогнозованим рейтингом 4.47


**Висновок:** виконавши дану лабораторну роботу, реалізувала два алгоритми для рекомендаційної системи на основі датасету BX-Book-Ratings.csv, використовуючи крос-валідацію для обох алгоритмів, та вивела рекомендації 10 книг для конкретного користувача.